In [1]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
import json
import ast
from urllib.parse import unquote

In [2]:
g = Graph()
g.parse("food_nutrition.ttl", format="turtle")

<Graph identifier=N3ee1cee06f8f4c048ff3a0398c6f4029 (<class 'rdflib.graph.Graph'>)>

In [3]:
queries = {
    "landing_page_groups.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT DISTINCT ?group
        WHERE {
          ?food a fn:Food ;
                fn:belongsToFoodWheelGroup ?group .
        }
        ORDER BY ?group
    """,

    "subgroups_overview.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT DISTINCT ?group ?subgroup
        WHERE {
          ?food a fn:Food ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup .
        }
        ORDER BY ?group ?subgroup
    """,

    "subpage_vegetables.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?diet 
               ?protein ?proteinUnit ?carbs ?carbsUnit ?fat ?fatUnit 
               ?vitaminC ?vitaminCUnit ?iron ?ironUnit
               ?benefit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToFoodWheelSubGroup fn:Vegetables .
          OPTIONAL { ?food fn:belongsToDiet ?diet }
          OPTIONAL { ?food fn:hasProtein ?protein }
          OPTIONAL { ?food fn:hasProteinUnit ?proteinUnit }
          OPTIONAL { ?food fn:hasCarbohydrates ?carbs }
          OPTIONAL { ?food fn:hasCarbohydratesUnit ?carbsUnit }
          OPTIONAL { ?food fn:hasFat ?fat }
          OPTIONAL { ?food fn:hasFatUnit ?fatUnit }
          OPTIONAL { ?food fn:hasVitaminC ?vitaminC }
          OPTIONAL { ?food fn:hasVitaminCUnit ?vitaminCUnit }
          OPTIONAL { ?food fn:hasIron ?iron }
          OPTIONAL { ?food fn:hasIronUnit ?ironUnit }
          OPTIONAL { ?food fn:hasBenefit ?benefit }
        }
        ORDER BY DESC(?vitaminC)
        LIMIT 30
    """,

    "top_vitaminC.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?vitaminC ?vitaminCUnit ?benefit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:hasVitaminC ?vitaminC ;
                fn:hasVitaminCUnit ?vitaminCUnit ;
                fn:hasBenefit ?benefit .
          FILTER(CONTAINS(STR(?benefit), "Vitamin C"))
        }
        ORDER BY DESC(?vitaminC)
        LIMIT 10
    """,

    "diet_filters_vegetarian.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?group ?subgroup 
               ?protein ?proteinUnit ?fat ?fatUnit ?carbs ?carbsUnit
               ?benefit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:belongsToDiet ?diet ;
                fn:belongsToFoodWheelGroup ?group ;
                fn:belongsToFoodWheelSubGroup ?subgroup .
          OPTIONAL { ?food fn:hasProtein ?protein }
          OPTIONAL { ?food fn:hasProteinUnit ?proteinUnit }
          OPTIONAL { ?food fn:hasFat ?fat }
          OPTIONAL { ?food fn:hasFatUnit ?fatUnit }
          OPTIONAL { ?food fn:hasCarbohydrates ?carbs }
          OPTIONAL { ?food fn:hasCarbohydratesUnit ?carbsUnit }
          OPTIONAL { ?food fn:hasBenefit ?benefit }
          FILTER(CONTAINS(STR(?diet), "Vegetarian"))
        }
        ORDER BY DESC(?protein)
        LIMIT 25
    """,

    "foods_potassium.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        SELECT ?foodName ?benefit
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:hasBenefit ?benefit .
          FILTER(CONTAINS(STR(?benefit), "Potassium"))
        }
        ORDER BY ?foodName
    """,

    "all_foods.json": """
        PREFIX fn: <http://example.org/food-nutrition#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        SELECT ?food ?foodName ?group ?subgroup ?diet ?benefit
              ?calories ?protein ?fat ?vitaminB12 ?vitaminC ?vitaminD ?iron
        WHERE {
          ?food a fn:Food ;
                fn:foodName ?foodName ;
                fn:hasCalories ?calories ;
                fn:hasProtein ?protein ;
                fn:hasFat ?fat ;
                fn:hasVitaminB12 ?vitaminB12 ;
                fn:hasVitaminC ?vitaminC ;
                fn:hasVitaminD ?vitaminD ;
                fn:hasIron ?iron .

          OPTIONAL { ?food fn:belongsToFoodWheelGroup ?group . }
          OPTIONAL { ?food fn:belongsToFoodWheelSubGroup ?subgroup . }
          OPTIONAL { ?food fn:belongsToDiet ?diet . }
          OPTIONAL { ?food fn:hasBenefit ?benefit . }
        }
"""
}

In [4]:
def row_to_dict(row):
    result = {}
    for k, v in row.asdict().items():
        if v is None:
            result[k] = None
        else:
            if isinstance(v, URIRef):
                result[k] = str(v)
            else:
                result[k] = str(v)
    return result

In [5]:
### Run queries and save results
for filename, query in queries.items():
    print(f"\nRunning query -> {filename}")
    results = g.query(query)
    data = [row_to_dict(row) for row in results]
    print(f"Rows: {len(data)}")

    if filename == "all_foods.json":
        foods = {}

        for row in data:
            food_uri = row["food"]
            if food_uri not in foods:
                foods[food_uri] = {
                    "foodName": unquote(row.get("foodName", "")),
                    "group": unquote(row.get("group", "").split("#")[-1]) if row.get("group") else None,
                    "subgroup": unquote(row.get("subgroup", "").split("#")[-1]) if row.get("subgroup") else None,
                    "diet": set(),       # use set instead of list
                    "benefit": set(),    # use set instead of list
                    "calories": float(row.get("calories", 0)),
                    "protein": float(row.get("protein", 0)),
                    "fat": float(row.get("fat", 0)),
                    "vitaminB12": float(row.get("vitaminB12", 0)),
                    "vitaminC": float(row.get("vitaminC", 0)),
                    "vitaminD": float(row.get("vitaminD", 0)),
                    "iron": float(row.get("iron", 0))
                }

            if row.get("diet"):
                diet_val = unquote(str(row["diet"]).split("#")[-1])
                if diet_val.startswith("["):
                    try:
                        parsed = ast.literal_eval(diet_val)
                        foods[food_uri]["diet"].update(parsed)  # use update() for sets
                    except:
                        foods[food_uri]["diet"].add(diet_val)
                else:
                    foods[food_uri]["diet"].add(diet_val)

            if row.get("benefit"):
                foods[food_uri]["benefit"].add(str(row["benefit"]))

        data = []
        for f in foods.values():
            f["diet"] = sorted(list(f["diet"]))
            f["benefit"] = sorted(list(f["benefit"]))
            data.append(f)

    with open("all_foods.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"Saved {len(data)} rows to {filename}")


Running query -> landing_page_groups.json
Rows: 5
Saved 5 rows to landing_page_groups.json

Running query -> subgroups_overview.json
Rows: 8
Saved 8 rows to subgroups_overview.json

Running query -> subpage_vegetables.json
Rows: 30
Saved 30 rows to subpage_vegetables.json

Running query -> top_vitaminC.json
Rows: 0
Saved 0 rows to top_vitaminC.json

Running query -> diet_filters_vegetarian.json
Rows: 25
Saved 25 rows to diet_filters_vegetarian.json

Running query -> foods_potassium.json
Rows: 0
Saved 0 rows to foods_potassium.json

Running query -> all_foods.json
Rows: 2339
Saved 1527 rows to all_foods.json


In [6]:
with open("all_foods.json", "r", encoding="utf-8") as f:
    foods = json.load(f)

ts_interface = """\
export interface Food {
  foodName: string;
  group: string | null;
  subgroup: string | null;
  diet: string[];
  benefit: string[];
  calories: number;
  protein: number;
  fat: number;
  vitaminB12: number;
  vitaminC: number;
  vitaminD: number;
  iron: number;
}

export const foods: Food[] = """

ts_data = json.dumps(foods, indent=2, ensure_ascii=False)

with open("all_foods.ts", "w", encoding="utf-8") as f:
    f.write(ts_interface + ts_data + ";\n")

print("Saved all_foods.ts successfully")

Saved all_foods.ts successfully
